In [1]:
import etl
import kaggle

In [2]:
kaggle.api.dataset_download_files("fedesoriano/electric-power-consumption", unzip=True)
file_input = etl.extraer_a_staging()

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/electric-power-consumption
Conexión exitosa
La base de datos 'proyect1_db' ya existe.
Datos cargados en la base de datos exitosamente.
Dataset almacenado en área de staging:
               datetime  temperature  humidity  wind_speed  \
0         1/1/2017 0:00        6.559      73.8       0.083   
1         1/1/2017 0:10        6.414      74.5       0.083   
2         1/1/2017 0:20        6.313      74.5       0.080   
3         1/1/2017 0:30        6.121      75.0       0.083   
4         1/1/2017 0:40        5.921      75.7       0.081   
...                 ...          ...       ...         ...   
52411  12/30/2017 23:10        7.010      72.4       0.080   
52412  12/30/2017 23:20        6.947      72.6       0.082   
52413  12/30/2017 23:30        6.900      72.8       0.086   
52414  12/30/2017 23:40        6.758      73.0       0.080   
52415  12/30/2017 23:50        6.580      74.1       0.081   

       general_diffuse_f